In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
from lxml import html
import pandas as pd

In [34]:
pages = list(range(1000,1399))

In [35]:
#取得文章標題
text_title = []
#取得文章連結
text_link = []
#取得文章內文
full_content = []
#取得文章留言
full_comment = []

for p in pages:
    board_url = "https://www.ptt.cc/bbs/facelift/index" + str(p) + ".html"
    re = requests.get(board_url)
    soup = BeautifulSoup(re.text, "html.parser")
    title = soup.findAll("div", {"class":"title"})
    link = []
    for i in range(len(title)):
        #忽略被刪除的文章
        try:
            link.append(title[i].a["href"])
            text_title.append(title[i].text)
        except:
            continue
    
    text_link.append(link)
        
    for i in range(len(link)):
        re_0 = requests.get("https://www.ptt.cc" + link[i])
        soup_0 = BeautifulSoup(re_0.text, "html.parser")
        #定位本文
        root = html.fromstring(re_0.text)
        tree = root.getroottree()
        content = root.xpath('//*[@id="main-content"]/text()')
        full_content.append(content)
        #定位留言
        c = soup_0.findAll("div", {"class":"push"})
        comment = []
        for i in c:
            comment.append(i.text)
        full_comment.append(comment)

In [36]:
print(len(text_title))
print(len(full_content))
print(len(full_comment))

7979
7979
7979


In [37]:
dic = {
    "title": text_title, 
    "content": full_content,
    "comment": full_comment,
}
df = pd.DataFrame(dic)
print(df)

                              title  \
0             \nFw: [請益] 醫美診所打玻尿酸\n   
1            \n[分享] 高雄中山一路某醫美態度不佳\n   
2               \n[問題] 吳外科還是老醫生嗎？\n   
3                     \n請問打淚溝的價位？\n   
4                 \n[問題] 抽脂消腫需要多久\n   
...                             ...   
7974            \nRe: [問題] 尋找提眼肌王\n   
7975          \n[問題] 皮秒雷射除刺青效果？？？\n   
7976         \n[問題] 高雄除斑選擇 高醫或榮總？\n   
7977  \n[心得] 割雙眼皮+提眼肌二個月心得分享~(文長)\n   
7978         \n[問題] Dr.V 葳亞娜診所 評價\n   

                                                content  \
0     [\n, \n作者: cocastar (K.Keith) 看板: ChangHua\n標題...   
1     [\n今天來跟大家閒聊一下這家位於高雄市新興區中山一路的XX紀醫美診所\n其門市小姐態度反應...   
2     [\n已預約2/14去縫雙眼皮，想請問最近有人去做嗎？還是老醫師嗎？因為版上最接近的日期\n...   
3     [\n請問大家知道台北或新北的價錢嗎？\n\n我上次問板橋光澤大概15000左右；但我聽說朋...   
4     [\n我在1/23那天抽脂\n\n大腿環抽(大腿前側沒抽)+馬鞍\n\n雖然瘀青跟痛的程度比...   
...                                                 ...   
7974  [\n就決定是你了！！睿醫-盧龍一\n\n昨天手術時間約兩個小時，順利的完成了\n，因為盧醫...   
7975  [\n\n手機排版，請見諒\n以前年輕的時候被愛沖昏頭，在腰上刺了一個彩色刺青，\n雖然已經...   


In [38]:
df1 = pd.DataFrame([pd.Series(x) for x in df.comment])
df1.columns = ['comment_{}'.format(x+1) for x in df1.columns]

<ipython-input-38-ed63e5cd1462>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df1 = pd.DataFrame([pd.Series(x) for x in df.comment])


In [40]:
df2 = pd.concat([df,df1],axis=1)

In [41]:
df2 = df2.drop(["comment"], axis=1)

In [42]:
df2.to_csv('/Users/linhengyu/Documents/Python practice/ccClub_project_1000-1399.csv', encoding='utf-8',index=False)